# Objectifs

L'objectif de l'exercice d'aujourd'hui est de vous familiariser avec les modèles de régression linéaire multivariables pour la prévision, et avec les concepts du *coefficient $R^2$* et du *ratio F* pour le test de Fisher, ou *F-test*. Lisez attentivement et comprenez les deux exemples au début du notebook, puis implémentez vous-même le modèle de régression et calculez les estimateurs statistiques pertinents en suivant les instructions dans chaque cellule qui suit.

# Modèles de prévision basés sur la régression

Les modèles de prévision basés sur la régression sont des exemples de modèles explicatifs construits en supposant qu'il existe une relation possible entre la variable à prévoir et un certain nombre de variables indépendantes.

Le terme **régression** fait référence à un certain type de modèle statistique qui tente de décrire la relation d'une variable, appelée **variable dépendante** (généralement notée par Y), et un certain nombre d'autres variables X₁, X₂, ..., Xₖ, appelées **variables explicatives** ou **indépendantes**. Nous ne considérerons que le cas d'une erreur additive, représentée par e, où la relation peut être écrite comme suit :

$$ Y = f(X₁, ..., Xₖ; b₀, b₁, ..., bₚ) + e $$

Ici :
- \(f\) est une fonction donnée connue sous le nom de **fonction de régression**.
- La fonction dépend de paramètres ou de coefficients, notés \(b₀, b₁, ..., bₚ\).
- Les valeurs des paramètres ne sont pas connues et doivent être estimées.
- Le nombre de paramètres de régression \(r = p + 1\) n'est pas nécessairement le même que \(k\).
- Il y a un élément incertain supplémentaire dans la relation, représenté par la variable aléatoire \(e\).
- La distribution de probabilité de \(e\) est généralement spécifiée, souvent supposée être normale (N(0, σ²)), mais avec la variance $(\sigma²)$ inconnue.

Les deux cellules ci-dessous vous montrent un exemple de régression linéaire multivariable pour certaines données bancaires contenues dans le fichier ``Bank.xls``, où vous prédisez les valeurs de la variable dépendante 'DEOM' à partir des trois variables indépendantes 'AAA', 'Tto4', 'D3to4'.

In [ ]:
from pandas import read_excel
from statsmodels.formula.api import ols
series = read_excel('Bank.xls',
                    sheet_name='Data2', header=0,
                     dtype=float)

# Lecture des variables de base
DEOM = series.DEOM
AAA = series.AAA
Tto4 = series.Tto4
D3to4 = series.D3to4

# Modèle(s) de régression
formula = 'DEOM ~ AAA + Tto4 + D3to4'

# Moindres carrés ordinaires (OLS)
results = ols(formula, data=series).fit()
print(results.summary())

# Ici, le tableau principal est le premier,
# où les principales statistiques sont le R-carré (ligne 1)
# et la valeur P ; c'est-à-dire, Prob (statistique F)

In [ ]:
from statsmodels.tsa.api import Holt
import matplotlib.pyplot as plt
import numpy as np

series = read_excel('Bank.xls',
                    sheet_name='Data2', header=0,
                      dtype=float)

# Lecture des variables de base
DEOM = series.DEOM
AAA = series.AAA
Tto4 = series.Tto4
D3to4 = series.D3to4

# Prévision pour AAA en utilisant la méthode linéaire de Holt
fit1 = Holt(AAA).fit(optimized=True)
fcast1 = fit1.forecast(6).rename("Additive 2 damped trend")
fit1.fittedvalues.plot(color='red')
fcast1.plot(color='red', legend=True)
AAA.plot(color='black', legend=True)
plt.title('Prévision de AAA avec la méthode linéaire de Holt')
plt.show()

# Prévision pour Tto4 en utilisant la méthode linéaire de Holt
fit2 = Holt(Tto4).fit(optimized=True)
fcast2 = fit2.forecast(6).rename("Additive 2 damped trend")
fit2.fittedvalues.plot(color='red')
fcast2.plot(color='red', legend=True)
Tto4.plot(color='black', legend=True)
plt.title('Prévision de 3to4 avec la méthode linéaire de Holt')
plt.show()

# Prévision pour D3to4 en utilisant la méthode linéaire de Holt
fit3 = Holt(D3to4).fit(smoothing_level=0.8, smoothing_trend=0.2, optimized=False)
#fit3 = Holt(D3to4).fit(optimized=True)
fcast3 = fit3.forecast(6).rename("Additive 2 damped trend")
fit3.fittedvalues.plot(color='red')
fcast3.plot(color='red', legend=True)
D3to4.plot(color='black', legend=True)
plt.title('Prévision de D3to4 avec la méthode linéaire de Holt')
plt.show()

# Construction de la prévision basée sur la régression pour la variable principale, DEOM
# Modèle(s) de régression
formula = 'DEOM ~ AAA + Tto4 + D3to4'

# ols génère des statistiques et les paramètres b0, b1, etc., du modèle
results = ols(formula, data=series).fit()
print(results.summary())
b0 = results.params.Intercept
b1 = results.params.AAA
b2 = results.params.Tto4
b3 = results.params.D3to4

# Mettre les valeurs ajustées des prévisions de AAA, Tto4, et D3to4 dans des tableaux
a1 = np.array(fit1.fittedvalues)
a2 = np.array(fit2.fittedvalues)
a3 = np.array(fit3.fittedvalues)

# Construction de la partie ajustée de la prévision de DEOM
F=a1
for i in range(53):
    F[i] = b0 + a1[i]*b1 + a2[i]*b2 + a3[i]*b3

# Mettre les valeurs des prévisions de AAA, Tto4, et D3to4 dans des tableaux
v1=np.array(fcast1)
v2=np.array(fcast2)
v3=np.array(fcast3)

# Construction des 6 valeurs de la prévision à l'avance
E=v1
for i in range(6):
    E[i] = b0 + v1[i]*b1 + v2[i]*b2 + v3[i]*b3


# Joindre les valeurs ajustées de la prévision et les points à l'avance
K=np.append(F, E)

# Lecture de la série temporelle DEOM originale pour toutes les 59 périodes
DEOMfull0 = read_excel('Bank.xls',
                       sheet_name='Data4', header=0,
                     dtype=float)


DEOMfull = DEOMfull0.DEOMfull

# Tracer les graphiques de K et DEOMfull avec des légendes associées.
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(K, color='red', label='Valeurs de prévision')
line2, = plt.plot(DEOMfull, color='black', label='Données originales')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=4)})
plt.title('Prévision de régression DEOM')
plt.show()

Complétez la cellule ci-dessus pour charger à nouveau la feuille 'Data2' du fichier 'Bank.xls' dans un dataframe pandas appelé 'series_2'.

Ensuite, chargez la variable dépendante 'DEOM_2', et les variables indépendantes 'AAA_2', 'Tto4_2', 'D3to4_2'.

Enfin, construisez et ajustez un modèle de régression pour prévoir la variable principale 'DEOM' comme vous l'avez vu dans l'exemple précédent, en imprimant le résumé des résultats à l'écran.

Le but de cette partie de l'exercice est que vous le fassiez vous-même à partir de zéro.

In [ ]:
series_2 = 
           
DEOM_2 = 
AAA_2 = 
Tto4_2 = 
D3to4_2 = 

#Modèle(s) de régression



Avec $Y$ représentant les valeurs observées, $\hat{Y}$ les estimations de $Y$ à partir de la régression (linéaire) et $\overline{Y}$ la moyenne des valeurs observées, rappelons :

**Somme totale des carrés, corrigée par la moyenne (SST)** :
$$
SST = \sum_{i=1}^{n} (Y_i-\overline{Y})^2
$$

**Somme des carrés de la régression (SSR)** :
$$
SSR = \sum_{i=1}^{n} (\hat{Y}_i-\overline{Y})^2
$$

**Somme des carrés résiduels, ou d'erreur (SSE)** :
$$
SSE = \sum_{i=1}^{n} (Y_i-\hat{Y}_i)^2
$$

Dans la cellule ci-dessous, calculez les estimations de 'DEOM' à partir de la régression linéaire ($\overline{Y}$ dans les formules ci-dessus) dans le tableau ``estimates``. Vous pouvez utiliser la méthode ``predict`` de statsmodel (<a href="https://www.statsmodels.org/devel/examples/notebooks/generated/predict.html">documentation</a>) ou les calculer manuellement comme pour F dans l'exemple précédent.

Comme exercice, vous pouvez utiliser les deux méthodes puis comparer les résultats pour vous assurer qu'ils correspondent.

In [ ]:
estimates = 

En utilisant le tableau ``estimates``, dans la cellule ci-dessous, calculez les SST, SSR, SSE pour les données de la banque que vous avez précédemment chargé.

In [ ]:
SST = 
SSR = 
SSE = 
print(SST, SSR, SSE)

Rappelez-vous que le **carré du coefficient de corrélation multiple** est défini comme : 

$R^2 = \frac{SSR}{SST} =1 - \frac{SSE}{SST}$

Calculez-le ci-dessous, et comparez-le avec la valeur retournée lorsque vous avez affiché à l'écran les résultats de la régression (vous pouvez également le récupérer en utilisant la méthode <a href="https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.rsquared.html">rsquared</a> dans statsmodel). Vous devriez obtenir le même résultat.

In [ ]:
R2 = 

Rappelez-vous que les **carrés moyens** sont définis, respectivement, comme suit :

**Total (MST)** : $$ MST = \frac{1}{n-1} \sum_{i=1}^{n} (Y_i-\overline{Y})^2 $$

**Expliqué (MSR)** : $$ MSR = \frac{1}{k} \sum_{i=1}^{n} (\hat{Y}_i-\overline{Y})^2 $$

**Inexpliqué (MSE)** : $$ MSE =\frac{1}{n-k-1} \sum_{i=1}^{n} (Y_i-\hat{Y}_i)^2 $$

où $n$ est le nombre d'observations, et $k$ est le nombre de coefficients dans la régression.

Calculez-les dans la cellule ci-dessous.

In [ ]:
k = 3
n = len(DEOM_2)
MST = 
MSR = 
MSE = 
print(MST, MSR, MSE)

Maintenant, calculez la valeur du ratio F pour le **test F de signification globale** (F-test of overall significance) :

$F = \frac{MSR}{MSE}$

et comparez-la avec la valeur imprimée dans le résumé des résultats de l'ajustement de la régression. Vous devriez obtenir la même valeur.

In [ ]:
F = 
print(F)